In [ ]:
import os
from astropy import units as u

from pocs.core import POCS
from pocs.utils import current_time
from huntsman.observatory import HuntsmanObservatory

## Setup POCS

In [ ]:
obs = HuntsmanObservatory(simulator=['weather'], take_flats=True, with_autoguider=True) # Or pass nothing as simulator

In [ ]:
pocs = POCS(obs, messaging=True, 
            state_machine_file='{}/conf_files/state_machine.yaml'.format(os.getenv('HUNTSMAN_POCS'))
           )

In [ ]:
pocs.initialize()

## Running Huntsman POCS

In [ ]:
pocs.run()

## Utility things below

### Test cameras

In [ ]:
import time
pocs.observatory.get_observation()

In [ ]:
pocs.observatory.current_observation.exp_time = 2 * u.s

In [ ]:
pocs.observatory.autofocus_cameras()

In [ ]:
for i in range(3):
    print("Exposure: ", i, current_time(pretty=True))
    events = pocs.observatory.observe()

    while not all([e.is_set() for e in events.values()]):
        pocs.sleep(3)

#### Optionally run coarse focusing

In [ ]:
cam_list = list(obs.cameras.keys())

In [ ]:
cam_list

In [ ]:
obs.autofocus_cameras(camera_list=cam_list, coarse=True)

### Experiment with Darks (ask Anthony)

In [ ]:
dark_path = '/var/huntsman/images/darks/83F010774/T-00C/300s'
n_darks = 19
for i in range(n_darks):
    filename = os.path.join(dark_path, "{}.fits".format(current_time(flatten=True)))
    obs.cameras['Cam00'].take_exposure(filename=filename, seconds=300, dark=True, blocking=True)

In [ ]:
dark_path = '/var/huntsman/images/darks/83F010774/T-00C/000s'
n_darks = 19
for i in range(n_darks):
    filename = os.path.join(dark_path, "{}.fits".format(current_time(flatten=True)))
    obs.cameras['Cam00'].take_exposure(filename=filename, seconds=0, dark=True, blocking=True)

In [ ]:
import time

In [ ]:
obs.cameras['Cam00'].set_point = -5

In [ ]:
time.sleep(300)

In [ ]:
dark_path = '/var/huntsman/images/darks/83F010774/T-05C/300s'
n_darks = 19
for i in range(n_darks):
    filename = os.path.join(dark_path, "{}.fits".format(current_time(flatten=True)))
    obs.cameras['Cam00'].take_exposure(filename=filename, seconds=300, dark=True, blocking=True)

In [ ]:
obs.cameras['Cam00'].set_point = -10

In [ ]:
time.sleep(300)

In [ ]:
dark_path = '/var/huntsman/images/darks/83F010774/T-10C/300s'
n_darks = 19
for i in range(n_darks):
    filename = os.path.join(dark_path, "{}.fits".format(current_time(flatten=True)))
    obs.cameras['Cam00'].take_exposure(filename=filename, seconds=300, dark=True, blocking=True)

In [ ]:
obs.cameras['Cam00'].set_point = 0

In [ ]:
del obs

In [ ]:
help(huntsman)

In [ ]:
events = pocs.observatory.autofocus_cameras(coarse=True)

In [ ]:
# exp test of exoplanet targets
pocs.observatory.get_observation()

In [ ]:
pocs.observatory.mount.set_target_coordinates(pocs.observatory.current_observation.field)

In [ ]:
pocs.observatory.mount.slew_to_target()

In [ ]:
i=0

In [ ]:
exp_time = 90
for cam_name, cam in pocs.observatory.cameras.items():
    fn = '/var/huntsman/images/temp/hatp27_{}_{}.fits'.format(cam_name, i)
    cam.take_exposure(seconds=exp_time, filename=fn)
    #fits.utils.get_solve_field(fn)
    i += 1

### Morning flats

In [ ]:
# wilfred tinkering below
pocs.observatory.mount.unpark()

In [ ]:
pocs.observatory.dome.connect()

In [ ]:
pocs.observatory.take_evening_flats(alt=70, az=300, max_exptime=120, camera_list=list(pocs.observatory.cameras.keys()))

In [ ]:
pocs.observatory.mount.park()

In [ ]:
pocs.observatory.dome.find_home()

In [ ]:
pocs.run()

In [ ]:
del obs